In [36]:
from streetview import search_panoramas
import folium
import numpy as np
# from geopy.distance import geodesic
from pyproj import Geod


In [37]:
def find_nearby(lat, lon, lats_ls, lons_ls):
  panos = search_panoramas(lat=lat, lon=lon)
  len(panos)


  # extract lat/lon from meta_ls
  for pano in panos:
      lats_ls.append(pano.lat)
      lons_ls.append(pano.lon)

  return lats_ls, lons_ls


  # Display map

In [38]:
center_lat, center_lon = 34.272, -118.484869
side_length = 100
geod = Geod(ellps='WGS84')

lats_ls, lons_ls = [], []

In [39]:

_, _, north_dist = geod.fwd(center_lon, center_lat, 0, side_length/2)
north_lat, north_lon = geod.fwd(center_lon, center_lat, 0, side_length/2)[1::-1]
east_lat, east_lon = geod.fwd(center_lon, center_lat, 90, side_length/2)[1::-1]

north = (north_lat, north_lon)
east = (east_lat, east_lon)
max_lat = north[0]  # Access first element of tuple for latitude
max_lon = east[1]   # Access second element of tuple for longitude
grid_size_lat = max_lat - center_lat
grid_size_lon = max_lon - center_lon

grid_size_lat, grid_size_lon

(0.000450745617506243, 0.0005429513391703722)

In [40]:
# Create grid cells based on radius_range
radius_range_meters = 3

# Using pyproj for more precise geodetic calculations

geod = Geod(ellps='WGS84')
_, _, north_dist = geod.fwd(center_lon, center_lat, 0, radius_range_meters)
north2_lat, north2_lon = geod.fwd(center_lon, center_lat, 0, radius_range_meters)[1::-1]
east2_lat, east2_lon = geod.fwd(center_lon, center_lat, 90, radius_range_meters)[1::-1]

north2 = (north2_lat, north2_lon)
east2 = (east2_lat, east2_lon)
max_lat2 = north2[0]  # Access first element of tuple for latitude
max_lon2 = east2[1]   # Access second element of tuple for longitude
radius_range_lat = max_lat2 - center_lat
radius_range_lon = max_lon2 - center_lon

radius_range_lat, radius_range_lon

(2.7044737990422618e-05, 3.2577080347095944e-05)

In [41]:
# grid_size = 0.001  # About 100m in each direction
grids = 10  # 10x10 grid

# num_points = grids/2
for i in range(int(-grids//2), int(grids//2) + 1):
    for j in range(int(-grids//2), int(grids//2) + 1):
        lat = center_lat + (i * grid_size_lat)
        lon = center_lon + (j * grid_size_lon)
        lats_ls, lons_ls = find_nearby(lat, lon, lats_ls, lons_ls)

KeyboardInterrupt: 

In [ ]:


# lats_sorted = sorted(lats_ls)
# lons_sorted = sorted(lons_ls)
# lat_min, lat_max = lats_sorted[0], lats_sorted[-1]
# lon_min, lon_max = lons_sorted[0], lons_sorted[-1]

# print(lat_min)
# print(lat_max)


34.27202704473799
0.000450745617506243
2.7044737990422618e-05
16.66666666826615


In [ ]:

# Calculate cell sizes
lat_cells = (lat_max - lat_min) // radius_range_lat + 1
lon_cells = (lon_max - lon_min) // radius_range_lon + 1

# Initialize grid to store points
grid_points = [[[] for _ in range(int(lat_cells))] for _ in range(int(lon_cells))]
print(f"Grid points shape: {len(grid_points)} x {len(grid_points[0])} cells")

# Categorize points into grid cells
for lat, lon in zip(lats_ls, lons_ls):
    # Calculate grid indices based on relative position
    lat_idx = int((lat - lat_min) // radius_range_lat) - 1
    lon_idx = int((lon - lon_min) // radius_range_lon) - 1
    
    # # Handle edge case where point is exactly at max boundary
    # if lat_idx == radius_range:
    #     lat_idx = lat_cells - 1
    # if lon_idx == radius_range:
    #     lon_idx = radius_range - 1
        
    # Only add if indices are valid
    # if 0 <= lat_idx < radius_range and 0 <= lon_idx < radius_range:
    #     grid_points[lat_idx][lon_idx].append((lat, lon))
    # print(lat_idx, " ", lon_idx)
    grid_points[lat_idx][lon_idx].append((lat, lon))


# # Create map centered on first point
# m = folium.Map(location=[lats_ls[0], lons_ls[0]], zoom_start=16)

# # Add markers for each point
# for lat, lon in zip(lats_ls, lons_ls):
#     folium.Marker([lat, lon]).add_to(m)


Grid points shape: 13 x 13 cells


In [ ]:
# Initialize list to store points and their neighbors within range
points_with_neighbors = []

# Helper function to calculate distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth's radius in meters
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Iterate through each cell in the grid
for i in range(len(grid_points)):
    for j in range(len(grid_points[0])):
        # Get points in current cell
        current_points = grid_points[i][j]
        
        # Skip if no points in current cell
        if not current_points:
            continue
            
        # Check adjacent cells (including diagonals)
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = i + di, j + dj
                
                # Skip if indices are out of bounds
                if not (0 <= ni < len(grid_points) and 0 <= nj < len(grid_points[0])):
                    continue
                    
                # Get points in neighbor cell
                neighbor_points = grid_points[ni][nj]
                
                # For each point in current cell
                for point1 in current_points:
                    # Compare with each point in neighbor cell
                    for point2 in neighbor_points:
                        # Skip if same point
                        if point1 == point2:
                            continue
                            
                        # Calculate actual distance
                        distance = haversine_distance(point1[0], point1[1], point2[0], point2[1])
                        
                        # If within range, add to results
                        if distance <= radius_range_meters:
                            points_with_neighbors.append((point1, point2, distance))

print(f"Found {len(points_with_neighbors)} point pairs within {radius_range_meters} meters")


Found 2506 point pairs within 3 meters


In [ ]:
grid_points

[[[(34.26971506066179, -118.4875065251815),
   (34.26969451004332, -118.4875468872872),
   (34.27006186334177, -118.4877103848993),
   (34.26999377251249, -118.4876268986729),
   (34.26992548072864, -118.4875435906034),
   (34.27008245426441, -118.4873522046335),
   (34.26985764391894, -118.4874606388904),
   (34.26978996094402, -118.4873783108333),
   (34.26972274333137, -118.4872966064332),
   (34.26972616223679, -118.4875831508042),
   (34.27008245426441, -118.4873522046335),
   (34.26985764391894, -118.4874606388904),
   (34.26978996094402, -118.4873783108333),
   (34.26972274333137, -118.4872966064332),
   (34.27013414736185, -118.4874151980848),
   (34.26985764391894, -118.4874606388904),
   (34.27013009761889, -118.4877942274813),
   (34.27006186334177, -118.4877103848993),
   (34.26999377251249, -118.4876268986729),
   (34.26992548072864, -118.4875435906034),
   (34.27006186334177, -118.4877103848993),
   (34.26999377251249, -118.4876268986729),
   (34.26992548072864, -118.4875

In [ ]:
grid_points[309][92]

IndexError: list index out of range